### Theory of bed profile forms implemented in some numpy functions
A bed profile model is required to estimate the bathymetry of a channel in 2D. Here we identify a set of functions to estimate the depth profile from a set of parameters that can be measured iun-situ or (ideally!) only at the side of the channel from drone footage. The angles at the sides of the channel are, according to Savenije 2003 and referenced authors, a good representation of bed form similarities. We therefore take these assumed relationships as starting point for our assessment.


In [ ]:
!pip install seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

## Replication of figure 1 in Savenije 2003

In [ ]:
# let's start with some coordinates from left (0) to right bank

def depth_from_width(s, phi=0.1*np.pi, h_m=10):
    """
    Simplest implementation of Eq. 12 from Savenije 2003, where the first coordinate (zero meters) is the lowest point. We stop at zero as that
    is where we reach the natural levee crest.
    """
    # zero is the lowest point in the stream starting in the middle of the stream
    angle = np.minimum(np.tan(phi)/h_m * s, 0.5*np.pi)
    depth = -np.cos(angle)*h_m
    return depth


s = np.linspace(0, 100, 1000)

depth = depth_from_width(s)

plt.plot(s, depth)
plt.xlabel("Distance from deepest point [m]")
plt.ylabel("Depth +shore [m]")


## Implementation over an entire symmetrical channel

Now we assume that the left coordinate is at the natural levee's left bank and the right at the natural levee's right bank.
We can now no longer use both `h_m` and `phi`. We have to resolve one of them to be able to solve the problem.

In [ ]:
def depth_profile_from_parameters(s, phi=None, h_m=None):
    """
    Solving the entire profile using a symmetrical assumption and either phi or h_m defined
    """
    if phi is None and h_m is None:
        raise ValueError("Either phi or h_m has to be supplied. You supplied nothing.")
    if phi is not None and h_m is not None:
        raise ValueError("Either phi or h_m has to be supplied. You supplied both. Select either one of the two")
    # compute the total width of the channel
    B = s[-1] - s[0]
    if phi is None:
        # h_m was supplied, so resolve for phi
        phi = np.arctan(np.pi*h_m/B)
    else:
        # phi was supplied, so resolve for h_m
        h_m = B*np.tan(phi)/np.pi
    # compute ordinate coordinates (i.e. from center line towards banks)
    _s = s - 0.5*B
    return depth_from_width(_s, phi=phi, h_m=h_m)
    



## Test over a set of natural angles of repose
Let's see what happens with different angles of repose. We should be assuming that milder angles lead to shallower water

In [ ]:

for n, phi in enumerate(np.linspace(0.05*np.pi, 0.25*np.pi, 10)):
    depth = depth_profile_from_parameters(s, phi=phi)
    plt.plot(s, depth, label="{:02d}: phi = {:1.2f}".format(n + 1, phi))
    plt.xlabel("Distance (left to right bank) [m]")
    plt.ylabel("Depth +shore [m]")
plt.axis("equal")
plt.legend(fontsize=8)

In [ ]:
# left bank
def depth_profile_left_right_parameters(s, phi_left, phi_right):
    depths_left = depth_profile_from_parameters(s, phi=phi_left)
    # right bank
    depths_right = depth_profile_from_parameters(s, phi=phi_right)
    weights = np.linspace(0, 1, len(s))
    return (1 - weights) * depths_left + weights * depths_right

s = np.linspace(0, 100, 100)
depths = depth_profile_left_right_parameters(s, phi_left=0.3*np.pi, phi_right=0.1*np.pi)
plt.plot(s, depths)
plt.axis("equal")
plt.xlabel("Distance (left to right bank) [m]")
plt.ylabel("Depth +shore [m]")
plt.title("asymmetrical profile shape")

In [ ]:
s = np.linspace(0, 100, 100000)
phi_left = 0.3*np.pi
phi_right = np.linspace(0.3*np.pi, 0.01*np.pi,100)
asym = []
for phi_r in phi_right:
    depths = depth_profile_left_right_parameters(s, phi_left=0.3*np.pi, phi_right=phi_r)
    idx = np.where(depths==depths.min())
    asym.append(s[idx[0][0]]/(s.max()-s[idx[0][0]]))



In [ ]:
s = np.linspace(0, 100, 100000)
phi_left2 = 0.1*np.pi
phi_right2 = np.linspace(0.3*np.pi, 0.01*np.pi,100)
asym2 = []
for phi_r in phi_right2:
    depths = depth_profile_left_right_parameters(s, phi_left=phi_left2, phi_right=phi_r)
    idx = np.where(depths==depths.min())
    asym2.append(s[idx[0][0]]/(s.max()-s[idx[0][0]]))


In [ ]:
s.max()-s[idx[0][0]]

In [ ]:
phi_rel2 =np.tan(phi_right2)/np.tan(phi_left2)
phi_rel2 =np.tan(phi_right2)/np.tan(phi_left2)
plt.plot(phi_rel, np.array(asym), ".", alpha=0.3)
plt.plot(phi_rel2, np.array(asym2), "r.", alpha=0.3)
plt.xlabel("tan phi / tan phi")
plt.ylabel("rel B")

### Asymmetrical channel behaviour
A channel bed is usually not entirely symmetrical, so we can also assume that some further constraints may be necessary to assess the width/depth relationship. If we supply the left and right angle, then this does not necessarily mean that the depth can be resolved from that, as they may to a certain degree counter. Hence we have to optimize the location over the cross section where the depth is at maximum.



### Equations

For the right side of the channel, the following applies:
$h_m=\frac{2B_r}{\pi}\tan{\phi_r}$

For left-wide, the following applies:

$h_m=\frac{2B_l}{\pi}\tan{\phi_l}$

Combining these two equations gives:

$\frac{\tan{\phi_r}}{\tan{\phi_l}}=\frac{B_l}{B_r}$

Meaning that if we know the angles of repose, we also know the relative distances from the deepest point. The only missing part is the depth in the deepest point, and then we known the absolute distances as well

### Modelling the location of the deepest point (distance from the middle of the grid) over the entire stretch
In a natural stream the location of the deepest point typically meanders with a higher frequency than the meanders of the natural levees. This means that the deepest point moves from the left to the right bank as the river travels downstream. This needs to be simulated in our bathymetric model.

Let's assume that we don't want to randomly pick the location of the deepest point, but instead we want to include it in the entire 2D spatial model of the depths. In this case we would need a longitudinal model for the location of the deepest point. We model it here as a distance from the centerline of the gridded domain, and assume here that the distance from the centerline can be modelled as a simple sine function as follows:

$y_{off}\left(x\right)=\bar{y_{off}} + A_y\sin{\left(2\pi\frac{x}{L}-\kappa\right)}$

where $y_{off}$ [m] is the offset from the datum (e.g. the left or right side of a grid, or the center line) we are looking for, simulated as function of the longitudinal distance $x$ [m], $A_y$ [m] is the amplitude of the offset, $L$ [m] is a length scale for the frequency angle of the function, and $\kappa$ [rad] the phase offset of the sinusoidal function. $\bar{y_{off}}$, $A_y$, $\kappa$ and $L$ are parameters that ideally are optimized with the available surveyed data points, along with the $\phi_{left}$ and $\phi_{right}$ parameters. We may choose to also constrain some of these parameters at some point, e.g. by allowing a user to say that they know where the dry/wet interface is located. That would fix a few of the parameters.

We implement the function below:



In [ ]:
def deepest_point_y_dist(x, y_off_mean, A_y, L, kappa):
    return y_off_mean + A_y*np.sin(2*np.pi*(x/L)-kappa)
                               
    

Let's try out the function interactively with a certain chosen domain

In [ ]:
from ipywidgets import interact
def plot_y_off(y_off_mean, A_y, L, kappa):  #y_off_mean, A_y, omega, kappa):
    x = np.linspace(0, 100, 10000)  # a 100 meter longitudinal stretch
    plt.plot(
        x, 
        deepest_point_y_dist(
            x,
            y_off_mean,
            A_y,
            L,
            kappa,
        )
    )
    plt.xlim([0, 100])
    plt.ylim([-50, 50])
            
interact(
    plot_y_off,
    y_off_mean=(-20, 20),
    A_y=(0, 40),
    L=(0, 200),
    kappa=(0, 2*np.pi)
)

### mapping the longitudinal function over a grid with distances from the middle
Let's assume we have a perfectly rectangular grid, where in each grid cell we have computed the distance of the grid cell with respect to one of the banks. the longitudinal diurection in the grid is from left to right, the transects are from top to bottom. We want to calculate for each grid cell, how far it is from the deepest point

In [ ]:
from matplotlib.colors import ListedColormap

def distance_to_deepest(xi, yi, y_off_mean, A_y, L, kappa):
    """
    Computes per grid cell, how far the deepest point is from zero coordinate in the y-direction
    """
    # first we compute for each coordinate where the deepest point in the given transect is with respect to the y=0 coordinate
    y_disti = deepest_point_y_dist(xi, y_off_mean, A_y, L, kappa)
    # then we compute how far in the transect we are per grid cell
    perp_distance = y_disti - yi
    return perp_distance

# Let's try it out with an interactive plot again

def plot_y_off_2d(y_off_mean, A_y, L, kappa):
    xax = np.linspace(0, 100, 1000)
    yax = np.linspace(0, 50, 500)
    xi, yi = np.meshgrid(xax, yax)  # xi are our longitudinal distances, yi our latitudinal distances
    perp_dist = distance_to_deepest(
        xi,
        yi,
        y_off_mean,
        A_y,
        L,
        kappa
    )
    palette = sns.color_palette("Spectral", 32) #, as_cmap=True)
    cmap = ListedColormap(colors=palette)
    plt.figure(figsize=(13, 6))
    plt.pcolormesh(
        xi,
        yi,
        perp_dist,
        cmap=cmap,
        vmin=-50,
        vmax=50,
    )        
    cb = plt.colorbar()
    cb.set_label("distance of deepest point from zero ordinal coordinate [m]")

    plt.xlabel("Longitudinal coordinate [m]")
    plt.ylabel("Ordinal coordinate [m]")
            
interact(
    plot_y_off_2d,
    y_off_mean=(0, 50),  # we limit the range from the minimum y-coordinate to the maximum
    A_y=(0, 40),
    L=(100, 500),
    kappa=(0, 2*np.pi)
)

### Final notes on deepest point model
The parameters of this model may need constraining, as we are gaining quite a lot of parameters along the way now. 

### Combining the ordinal model with the longitudinal model
We now have a means to estimate the location of the deepest point, now we need to estalbish a 2D model that uses both concepts to estimate bathymetry. Here we can introduce a new constraint, namely that the conveyance from bank to bank should not change significantly from transect to transect.

